In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import importlib
import pandas as pd
import seaborn
from IPython.display import Image
import matplotlib.pyplot as plt

import Transformer as tnsf
import preprocess as ad

importlib.reload(ad)
importlib.reload(tnsf)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
pd.options.display.width = 400
pd.options.display.max_colwidth = 400

## Data Preproccesing

In [10]:
# Parse from file
# Input log file name
log_file  = 'HDFS_2k.log'
log_source  = 'HDFS'
algorithm = 'Spell'
ad.parse(log_source, log_file, algorithm)

Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.412718]


In [11]:
# Original raw logs
N = 10
with open("Dataset/HDFS/HDFS.log") as file:
    for i in range(N): print(next(file).strip())

081109 203518 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010
081109 203518 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906
081109 203519 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.10.6:40524 dest: /10.250.10.6:50010
081109 203519 145 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.14.224:42420 dest: /10.250.14.224:50010
081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_-1608999687919862906 terminating
081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_-1608999687919862906 terminating
081109 203519 145 INFO dfs.DataNode$PacketResponder: Received block blk_-1608999687919862906 of size 91178 from /10.250.10.6
081109 203519 145 INFO dfs.DataNode$PacketResponder: Receiv

In [20]:
# Identified log patterns
log_structured = pd.read_csv("Spell_results/HDFS.log_templates.csv") 
log_structured

Log Key                                                                                                                                                                        Message  Occurrences
0         2                                                                                                                                           Receiving block <*> src <*> dest <*>      1723232
1         6                                                                                                 BLOCK* NameSystem.addStoredBlock blockMap updated <*> is added to <*> size <*>      1719741
2         5                                                                                                                                             Received block <*> of size <*> <*>      1713611
3         4                                                                                                                                          PacketResponder <*> for block <*> <*>      1706728
4        14                                                                                                                                                    Deleting block <*> file <*>      1402047
5        24                                                                                                                     BLOCK* NameSystem.delete <*> is added to invalidSet of <*>      1396174
6         3                                                                                                                                        BLOCK* NameSystem.allocateBlock <*> <*>       575061
7         7                                                                                                                                                           <*> block <*> to <*>       435697
8        26                                                                                                                                      <*>Got exception while serving <*> to <*>       356207
9        10                                                                                                                                                 Verification succeeded for <*>       120036
10        8                                                                                                                           <*> Starting thread to transfer block <*> to <*> <*>         7002
11        9                                                                                                                          BLOCK* ask <*> to replicate <*> to datanode s <*> <*>         7002
12       25                                                                                                 Unexpected error trying to delete block <*>. BlockInfo not found in volumeMap.         5545
13       11                                                                                                                                          writeBlock <*> received exception <*>         3300
14       15                                                                                                                                           Receiving empty packet for block <*>         1464
15       18                                                                             BLOCK* NameSystem.addStoredBlock Redundant addStoredBlock request received for <*> on <*> size <*>          975
16       13                                                                                               writeBlock <*> received exception java.io.IOException Could not read from stream           82
17       16                                                                                           Exception in receiveBlock for block <*> java.io.IOException Connection reset by peer           75
18       20                                                                                                Changing block file offset of block <*> from <*> to <*> meta file offset to <*>           65
19  

## Log sequences are divided based on each unique block_id
## Block_ids represent different sessions of log events related to each other

In [21]:
# Log sequences 
# Rows are divided based on each unique block_id
# We create a new row when there is a new block_id in the structured event logs
N = 10
with open("Dataset/HDFS/hdfs_train") as file:  # the a opens it in append mode
    for i in range(N):
        print(next(file).strip())

5 5 5 22 11 9 11 9 11 9 26 26 26 23 23 23 21 21 21
22 5 5 5 11 9 11 9 11 9 26 26 26
22 5 5 5 26 26 26 11 9 11 9 11 9 2 3 23 23 23 21 21 21
22 5 5 5 11 9 11 9 11 9 26 26 26
22 5 5 5 26 26 26 11 9 11 9 11 9 4 3 3 3 4 3 4 3 3 4 3 3 23 23 23 21 21 21
22 5 5 5 26 26 26 11 9 11 9 11 9 3 3 4 3 4 3 3 3 4 4 3 3 23 23 23 21 21 21
5 22 5 5 26 26 11 9 11 9 11 9 26 23 23 23 21 21 21
22 5 5 5 26 26 26 11 9 11 9 11 9 4 4 3 2 23 23 23 21 21 21
5 22 5 5 11 9 11 9 11 9 26 26 26 23 23 23 21 21 21
5 5 5 22 11 9 11 9 11 9 26 26 26 23 23 23 21 21 21


## Model Training

In [14]:
import argparse
import sys

sys.argv = ['']
parser = argparse.ArgumentParser()

parser.add_argument('--log_file', default='HDFS/hdfs_train', type=str, help='parsed log file')
parser.add_argument('--log_normal', default='HDFS/hdfs_test_normal', type=str, help='parsed log file of normal testing data')
parser.add_argument('--log_abnormal', default='HDFS/hdfs_test_abnormal', type=str, help='parsed log file of abnormal testing data')

parser.add_argument('--window_size', default=10, type=int, help='lenght of training window')

parser.add_argument('--batch_size', default=512, type=int, help='input batch size for training')
parser.add_argument('--epochs', default=10, type=int, help='number of epochs to train')

parser.add_argument('--dropout', default=0.2, type=float, help='number of epochs to train')
parser.add_argument('--num_layers', default=1, type=int, help='number of encoder and decoders')
parser.add_argument('--num_heads', default=1, type=int, help='number of heads')
parser.add_argument('--seed', default=1, type=int, help='random seed')

parser.add_argument('--num_classes', default =29, type=int, help='number of total log keys')
parser.add_argument('--num_candidates', default=10, type=int, help='number of predictors sequence as correct predict')

parser.add_argument('--federated', default=False, type=bool, help='number of gpus of gpus to train')      
parser.add_argument('--num_gpus', default=1, type=int, help='number of gpus of gpus to train')
parser.add_argument('--model_dir', default='Model', type=str, help='the directory to store the model')
parser.add_argument('--data_dir', default='Dataset/', type=str, help='the directory where training data is stored')

args = parser.parse_args()

_StoreAction(option_strings=['--data_dir'], dest='data_dir', nargs=None, const=None, default='Dataset/', type=<class 'str'>, choices=None, help='the directory where training data is stored', metavar=None)

In [16]:
model = tnsf.train(args)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Sessions 46575


C:\ProgramData\anaconda3\lib\site-packages\torch\nn\parallel\_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch Step: 1 Loss: 3.817143 Tokens per Sec: 6948.968262
Epoch Step: 51 Loss: 0.946231 Tokens per Sec: 40468.000000
Number of sessions(Dataset/HDFS/hdfs_test_normal): 14177
Number of sessions(Dataset/HDFS/hdfs_test_abnormal): 4123


 10%|████████▎                                                                          | 1/10 [00:27<04:06, 27.43s/it]

...

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [03:46<00:57, 28.50s/it]

Epoch Step: 1 Loss: 0.282745 Tokens per Sec: 36916.996094
Epoch Step: 51 Loss: 0.298923 Tokens per Sec: 40009.753906
Number of sessions(Dataset/HDFS/hdfs_test_normal): 14177
Number of sessions(Dataset/HDFS/hdfs_test_abnormal): 4123


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [04:15<00:28, 28.62s/it]

Epoch Step: 1 Loss: 0.298570 Tokens per Sec: 34036.003906
Epoch Step: 51 Loss: 0.278906 Tokens per Sec: 40211.468750
Number of sessions(Dataset/HDFS/hdfs_test_normal): 14177
Number of sessions(Dataset/HDFS/hdfs_test_abnormal): 4123


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:44<00:00, 28.43s/it]


## Model evaluation

In [17]:
parser = argparse.ArgumentParser()

parser.add_argument('--log_normal', default='HDFS/hdfs_test_normal', type=str, help='parsed log file of normal testing data')
parser.add_argument('--log_abnormal', default='HDFS/hdfs_test_abnormal', type=str, help='parsed log file of abnormal testing data')

parser.add_argument('--window_size', default=10, type=int, help='lenght of training window')
parser.add_argument('--num_candidates', default=10, type=int, help='number of candidates considered correct predict')

parser.add_argument('--federated', default=False, type=bool, help='number of gpus of gpus to train')
parser.add_argument('--num_gpus', default=0, type=int, help='number of gpus of gpus to train')
parser.add_argument('--model_dir', default='Model', type=str, help='the directory to store the model')
parser.add_argument('--data_dir', default='Dataset', type=str, help='the directory where training data is stored')

args = parser.parse_args()

tnsf.test(args)

Number of sessions(Dataset\HDFS/hdfs_test_normal): 14177
Number of sessions(Dataset\HDFS/hdfs_test_abnormal): 4123


## Visualization

In [6]:
# We create log sequences with 10 log keys
WINDOW_SIZE = 10
importlib.reload(tnsf)

<module 'Transformer' from 'C:\\Users\\Luis Selvera\\Desktop\\Anomaly_Detection_Transformer\\Transformer.py'>

In [8]:
model = torch.load("Model/centralized_models.pt")
model.eval()

# Parameter
WINDOW_SIZE = 10

# Log sequence
seq = [22, 5, 5, 5, 26, 26, 26, 11, 9, 11, 9, 11, 9, 3, 3, 4, 3, 4, 3, 3]

# Input log sequence
src = seq[:WINDOW_SIZE]
# Output log sequence
tgt = seq[WINDOW_SIZE:]

src_mask = Variable(torch.ones(1, 1, WINDOW_SIZE + 1)).to(device)
bos = torch.ones((1, ),dtype = int).to(device)

t1 = torch.cat((bos, torch.tensor(src, dtype = torch.int).to(device))).unsqueeze(0)
t2 = torch.tensor(tgt, dtype = torch.int).to(device).unsqueeze(0)

src = Variable(t1, requires_grad =False)
tgt = Variable(t2, requires_grad =False)

# Predicted log sequence
tgt_pred = tnsf.greedy_decode(model, src, src_mask, tgt, WINDOW_SIZE + 1, 1, True, g=10, halt = False, layers = 1, heads = 1)

^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(9, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([22, 15,  8, 27, 16,  5, 21, 26,  9, 11], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(11, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([20, 22, 21, 27, 16, 18,  5, 26,  9, 11], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(9, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([ 8, 22, 23, 19, 15, 27, 25, 26, 11,  9], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(3, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([27, 28, 25, 18, 11, 26,  2, 23,  4,  3], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~

In [9]:
tgt = np.reshape(tgt.cpu().detach().numpy(), (10,))
tgt = list(tgt)
src = list(np.reshape(src.cpu().detach().numpy(), (11,)))

In [10]:
def draw(data, x, y, ax, cbar_ax):
    seaborn.heatmap(data, xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0,annot=False, cbar_ax=cbar_ax,cbar=True, ax=ax)

### Horizontal axis represents the logs the model is predicting. 
### Vertical axis are the logs that the model is focusing on, applying attention, to predict next log on horizontal axis.

In [ ]:
heads = 1
layers = 1
for layer in range(layers):
    fig, axs = plt.subplots(1,layers, figsize=(20, 10))
    print("Encoder Layer", layer+1)
    for h in range(heads):
        draw(model2.encoder.layers[layer].self_attn.attn[0, h].data, 
            sent, sent if h ==0 else [], ax=axs[h])
    plt.show()
    
for layer in range(layers):
    fig, axs = plt.subplots(1,layers, figsize=(20, 10))
    print("Decoder Self Layer", layer+1)
    for h in range(4):
        draw(model2.decoder.layers[layer].self_attn.attn[0, h].data[:len(tgt_sent), :len(tgt_sent)], 
            tgt_sent, tgt_sent if h ==0 else [], ax=axs[h])
    plt.show()
    print("Decoder Src Layer", layer+1)
    fig, axs = plt.subplots(1,4, figsize=(20, 10))
    for h in range(heads):
        draw(model2.decoder.layers[layer].self_attn.attn[0, h].data[:len(tgt_sent), :len(sent)], 
            sent, tgt_sent if h ==0 else [], ax=axs[h])
    plt.show()

|  | <font size="4">Positive</font>  | <font size="4">Negative</font> |
| --- | --- | --- |
| <font size="4">**True**</font> | <font size="4">There is a problem and alarm turns on </font>| <font size="4">There is no problem and no alarm </font>|
| <font size="4">**False**</font> | <font size="4">There is no problem but alarm turns on</font> | <font size="4">There is a problem but no alarm</font> |

## Normal behavior log sequence

In [11]:
# Log sequence
seq = [22, 5, 5, 5, 26, 26, 26, 11, 9, 11, 9, 11, 9, 3, 3, 4, 3, 4, 3, 3]

# Input log sequence
src = seq[:WINDOW_SIZE]
# Output log sequence
tgt = seq[WINDOW_SIZE:]

src_mask = Variable(torch.ones(1, 1, WINDOW_SIZE + 1)).to(device)
bos = torch.ones((1, ),dtype = int).to(device)

t1 = torch.cat((bos, torch.tensor(src, dtype = torch.int).to(device))).unsqueeze(0)
t2 = torch.tensor(tgt, dtype = torch.int).to(device).unsqueeze(0)

src = Variable(t1, requires_grad =False)
tgt = Variable(t2, requires_grad =False)

# Predicted log sequence
tgt_pred = tnsf.greedy_decode(model, src, src_mask, tgt, WINDOW_SIZE + 1, 1, True, g=10, halt = True, layers = 1, heads = 1)

if -1 in tgt_pred: print("Abnormal log sequence: false positive")
else: print("Normal log sequence: true negative\n", tgt_pred)

^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(9, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([22, 15,  8, 27, 16,  5, 21, 26,  9, 11], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(11, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([20, 22, 21, 27, 16, 18,  5, 26,  9, 11], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(9, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([ 8, 22, 23, 19, 15, 27, 25, 26, 11,  9], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(3, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([27, 28, 25, 18, 11, 26,  2, 23,  4,  3], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~

## Abnormal behavior log sequence

In [15]:
#Log sequence

seq = [25, 18, 5, 16, 6, 26, 26, 21, 3, 3, 4, 23, 23, 23, 21, 21, 21]

# Input log sequence
src = seq[:WINDOW_SIZE]
# Output log sequence
tgt = seq[WINDOW_SIZE:]

src_mask = Variable(torch.ones(1, 1, WINDOW_SIZE + 1)).to(device)
bos = torch.ones((1, ),dtype = int).to(device)

t1 = torch.cat((bos, torch.tensor(src, dtype = torch.int).to(device))).unsqueeze(0)
t2 = torch.tensor(tgt, dtype = torch.int).to(device).unsqueeze(0)

src = Variable(t1, requires_grad =False)
tgt = Variable(t2, requires_grad =False)

# Predicted log sequence
tgt_pred = tnsf.greedy_decode(model, src, src_mask, tgt, WINDOW_SIZE + 1, 1, True, g=10, halt = True, layers = 1, heads = 1) 

#If -1 in predicted sequence, anomaly detected
if -1 in tgt_pred: 
    print("Abnormal log sequence: true positive\n", tgt_pred)
#Otherwise, no anomaly was detected
else: 
    print("Normal log sequence: false negative")

^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Incoming log: tensor(4, device='cuda:0', dtype=torch.int32)
Candidate logs:  tensor([22, 15, 10, 25, 18, 21,  5, 16,  6, 26], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
Abnormal log sequence: true positive
 tensor([[ 4, -1]], device='cuda:0')


## Find logs that caused anomaly

In [22]:
tgt_pred

tensor([[ 4, -1]], device='cuda:0')

In [30]:
importlib.reload(ad)

log_source = "HDFS"
algorithm = "Spell"
#Backtrack to logs that caused anomaly
ad.backtrace(tgt_pred[0], log_source, algorithm)

4 3    PacketResponder <*> for block <*> <*>
Name: Message, dtype: object


## Mapping between log key and event templates

In [32]:
#Mapping between log key and event template
log_structured = pd.read_csv("Spell_results/" +  "HDFS.log_templates.csv") 
log_structured

Log Key                                            Message  Occurrences
0         2               Receiving block <*> src <*> dest <*>      1723232
1         6  BLOCK* NameSystem.addStoredBlock blockMap upda...      1719741
2         5                 Received block <*> of size <*> <*>      1713611
3         4              PacketResponder <*> for block <*> <*>      1706728
4        14                        Deleting block <*> file <*>      1402047
5        24  BLOCK* NameSystem.delete <*> is added to inval...      1396174
6         3            BLOCK* NameSystem.allocateBlock <*> <*>       575061
7         7                               <*> block <*> to <*>       435697
8        26          <*>Got exception while serving <*> to <*>       356207
9        10                     Verification succeeded for <*>       120036
10        8  <*> Starting thread to transfer block <*> to <...         7002
11        9  BLOCK* ask <*> to replicate <*> to datanode s ...         7002
12       25  Unexpected error trying to delete block <*>. B...         5545
13       11              writeBlock <*> received exception <*>         3300
14       15               Receiving empty packet for block <*>         1464
15       18  BLOCK* NameSystem.addStoredBlock Redundant add...          975
16       13  writeBlock <*> received exception java.io.IOEx...           82
17       16  Exception in receiveBlock for block <*> java.i...           75
18       20  Changing block file offset of block <*> from <...           65
19       19        Exception in receiveBlock for block <*> <*>           59
20       30      PendingReplicationMonitor timed out block <*>           47
21       12              PacketResponder <*> <*> Exception <*>           44
22       32               Adding an already existing block <*>           10
23       23  <*>Failed to transfer <*> to <*> got java.io.I...            9
24       22  writeBlock <*> received exception java.io.IOEx...            9
25       28                                   Reopen Block <*>            5
26       31  PacketResponder <*> <*> Exception java.io.Inte...            5
27       29  BLOCK* Removing block <*> from neededReplicati...            4
28       17  writeBlock <*> received exception java.io.IOEx...            3
29       27  PacketResponder <*> <*> Exception java.io.IOEx...            2
30       21  PacketResponder <*> <*> Exception java.io.IOEx...            2